# Libraries


In [6]:
import numbers
from numpy.lib.stride_tricks import as_strided
import numpy as np
from skimage import io, util
import sklearn.feature_extraction as sk
# from ksvd import ApproximateKSVD
import cv2
from itertools import product
import matplotlib.pyplot as plt
import pickle

# KSVD CLASS 

## Ksvd class for training the dictionary.

In [7]:
# coding:utf-8
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.linear_model import orthogonal_mp_gram
import math


class ApproximateKSVD(object):
    def __init__(self, n_components, max_iter=1, diction = None, tol=1e-6,
                 transform_n_nonzero_coefs = None):
        """
        Parameters
        ----------
        n_components:
            Number of dictionary elements

        max_iter:
            Maximum number of iterations

        tol:
            tolerance for error

        transform_n_nonzero_coefs:
            Number of nonzero coefficients to target
        """
        self.components_ = None
        self.max_iter = max_iter
        self.tol = tol
        self.n_components = n_components
        self.transform_n_nonzero_coefs = transform_n_nonzero_coefs
        self.D = diction

    def _update_dict(self, X, D, gamma):
        for j in range(self.n_components):
            I = gamma[:, j] > 0
            if np.sum(I) == 0:
                continue

            D[j, :] = 0
            g = gamma[I, j].T
#             print(gamma.shape)
#             print(X.shape)
#             print(len(I))
            r = X[I, :] - gamma[I, :].dot(D)
            d = r.T.dot(g)
            d /= np.linalg.norm(d)
            g = r.dot(d)
            D[j, :] = d
            gamma[I, j] = g.T
        return D, gamma

    def _initialize(self, X):
        if min(X.shape) < self.n_components:
            D = np.random.randn(self.n_components, X.shape[1])
#             print("Dictionary:", D.shape)
        else:
            u, s, vt = sp.sparse.linalg.svds(X, k=self.n_components)
            D = np.dot(np.diag(s), vt)
        D /= np.linalg.norm(D, axis=1)[:, np.newaxis]
        return D

    def _transform(self, D, X):
        gram = D.dot(D.T)
#         print("Gamma:", gram.shape)
        Xy = D.dot(X.T)
#         print("Xy:", Xy.shape)

        n_nonzero_coefs = self.transform_n_nonzero_coefs
        if n_nonzero_coefs is None:
            n_nonzero_coefs = int(0.1 * D.shape[0])
            print("non-zero", n_nonzero_coefs)
        else:
            print("Non-zero", n_nonzero_coefs)

        return orthogonal_mp_gram(
            gram, Xy, n_nonzero_coefs=n_nonzero_coefs).T


    def fit(self, X):
        """
        Parameters
        ----------
        X: shape = [n_samples, n_features]
        """
        # print(X)
        # print(X.shape[1])
        y = []
        avg = []
        # D = self.D
        D = self._initialize(X)
        transform_n_nonzero_coefs = self.transform_n_nonzero_coefs
        for i in range(self.max_iter):
            gamma = self._transform(D, X)
            e = np.linalg.norm(X - gamma.dot(D))
            y.append(e)
            print("||Y - DX||2 for iteration {0} is: ".format(i+1), e)
            if e < self.tol:
                break
            D, gamma = self._update_dict(X, D, gamma)

        plt.plot(y)
        with open("sigma_10_0.3.txt", "wb") as fp:   #Pickling
            pickle.dump(y, fp)
        plt.show()

        self.components_ = D

        return self

    def transform(self, X):
        return self._transform(self.components_, X)


# Function to compute the number of patches

In [9]:
def _compute_n_patches(i_h, i_w, p_h, p_w, step, max_patches=None):

    n_h = (i_h - p_h) // step + 1
    n_w = (i_w - p_w) // step + 1
    all_patches = n_h * n_w

    return all_patches

# Function to extract the patches from Input Image

In [10]:
def _extracting(arr, patch_shape=8, extraction_step=1):

    arr_ndim = arr.ndim

    if isinstance(patch_shape, numbers.Number):
        patch_shape = tuple([patch_shape] * arr_ndim)
    if isinstance(extraction_step, numbers.Number):
        extraction_step = tuple([extraction_step] * arr_ndim)

    patch_strides = arr.strides

    slices = tuple(slice(None, None, st) for st in extraction_step)
    indexing_strides = arr[slices].strides

    patch_indices_shape = ((np.array(arr.shape) - np.array(patch_shape)) //
                           np.array(extraction_step)) + 1

    shape = tuple(list(patch_indices_shape) + list(patch_shape))
    strides = tuple(list(indexing_strides) + list(patch_strides))

    patches = as_strided(arr, shape=shape, strides=strides)

    # patches = patches.reshape([-1] + list(patch_shape))

    return patches

# Function to call the extraction step and reshaping the patches

In [11]:
def extract_patches(image, patch_size, step, max_patches=None, random_state=None):

    i_h, i_w = image.shape[:2]
    p_h, p_w = patch_size

    # image = check_array(image, allow_nd=True)
    image = image.reshape((i_h, i_w, -1))
    n_colors = image.shape[-1]

    extracted_patches = _extracting(image, patch_shape=(p_h, p_w, n_colors), extraction_step=step) # above function call
    n_patches = _compute_n_patches(i_h, i_w, p_h, p_w, step, max_patches)
    # print n_patches
    # if max_patches:
    #     rng = check_random_state(random_state)
    #     i_s = rng.randint(i_h - p_h + step, size=n_patches)
    #     j_s = rng.randint(i_w - p_w + step, size=n_patches)
    #     patches = extracted_patches[i_s, j_s, 0]
    # else:
    patches = extracted_patches

    patches = patches.reshape(-1, p_h, p_w, n_colors)
    # print patches.shape
    # remove the color dimension if useless
    if patches.shape[-1] == 1:
        return patches.reshape(n_patches, p_h, p_w)
    else:
        return patches


# Funtion for Reconstruction of the image from the patches

In [12]:
def reconstruct_patches(patches, image_size, step):

    i_h, i_w = image_size[:2]
    p_h, p_w = patches.shape[1:3]
    img = np.zeros(image_size)
    # compute the dimensions of the patches array
    n_h = int((i_h - p_h) / step + 1)
    n_w = int((i_w - p_w) / step + 1)
    for p, (i, j) in zip(patches, product(range(n_h), range(n_w))):
        img[i * step:i * step + p_h, j * step:j * step + p_w] += p

    for i in range(i_h):
        for j in range(i_w):
            img[i, j] /= float(min(i + step, p_h, i_h - i) *
                               min(j + step, p_w, i_w - j))
    return img


# If the input image is not noisy, function to add noise.

In [ ]:
img = util.img_as_float(io.imread("C:/Users/DELL/PycharmProjects/ksvd_final/input/Lenna.jpg"))
print("Adding Noise")
img_with_noise = util.random_noise(img, seed=1)
io.imsave("noise.png", img_with_noise)
print("Done with noise")

# Loading of Image and Extracting patches

In [ ]:
img = util.img_as_float(cv2.imread("girl_face_noise_15.jpg", 0))
# image = io.imread("C:/Users/DELL/PycharmProjects/ksvd_final/Prova.jpg")

res = extract_patches(img, (8, 8), 1)
# width, height, col= res.shape
# print("initial Image shape", img.shape)
print("Patches shape: ", res.shape)
num_patch, p_h, p_w = res.shape

# Functions for converting 2d or 3d matrix to vector and vice versa

In [24]:
def matrix_to_vector(matrix, patch_size):
    
    list = np.split(matrix.reshape(-1,patch_size),matrix.shape[0])
    
    list1 = []
    
    
    for arr in list:
        bb = arr.flatten('F').reshape(patch_size**2,1)
        list1.append(bb)

    array = np.array(list1).squeeze()
    
    return array


def vector_to_matrix_2d(vector,h,w):
    
    ab = np.reshape(vector,(h,w))
    return ab.T

def vector_to_matrix_3d(vector, h, w, col):

    ab = np.reshape(vector,(h,w,col))
#     print(ab.T)
    return ab.transpose(0,2,1)


# aa= np.array(range(27)).reshape(3,3,3)
# print(aa)
# print("------------------------")

# bb = matrix_to_vector(aa, 3)
# print(bb)

# ab = vector_to_matrix_3d(array, -1,3,3)
# print(ab)

# ba = vector_to_matrix_2d(bb[0], 3, 3)
# print(ba)




# Pre-Processing step: Removing the mean from each value of the patches

In [ ]:
imag = res[1]
print(imag.shape)
io.imshow(imag, cmap= 'gray')
io.show()

signals = matrix_to_vector(res, p_w)

num_patches, patch_size = signals.shape

print("Signals final shape", signals.shape)
mean = np.mean(signals, axis=0)[:, np.newaxis]
print("Mean value shape: ", mean.shape)

mean1 = vector_to_matrix_2d(mean,p_h,p_w)
io.imshow(mean1, cmap = 'gray')
io.show()

signals = (signals.T - mean).T

signals1 = vector_to_matrix_2d(signals[1], p_h, p_w)
io.imshow(signals1, cmap = 'gray')
io.show()

signal_final = signals1 + mean1

io.imshow(signal_final,  cmap= 'gray')
io.show()

if signal_final.all() == imag.all():
    print("True")
else:
    print("false")

# Applying KSVD

In [ ]:
import scipy.io as sio

dict_list = []
gamma_list = []

# bdict = sio.loadmat('dictionary_intialize.mat') (Predefined Dictionary)
# for rel in bdict:
#    if rel == 'dictionary':
#       diction = bdict['dictionary']

# print(diction.shape)

print("Applying KSVD")
aksvd = ApproximateKSVD(n_components=128, transform_n_nonzero_coefs = 10)
dictionary = aksvd.fit(signals[:10000]).components_
#dictionary = aksvd.fit(signals)
print("Done with learning")
# np.savetxt("test.txt", dictionary)
gamma = aksvd.transform(signals)
print("Found gamma")

# import scipy.io as sio 

# adict = {}
# adict['dictionary'] = dictionary

# sio.savemat('dictionary_Coeff_{}.mat'.format(30), adict) % Save the trained dictionary

# bdict = sio.loadmat('dictionary_Coeff_{}.mat'.format(30))
# for rel in bdict:
#     print(rel)

In [ ]:
print(dictionary[1].shape)
element = dictionary[1] + mean.T
print(element.shape)
element_1 = vector_to_matrix_2d(element, p_h, p_w)
io.imshow(element_1, cmap = 'gray')
io.show()

# Function for extracting images (patches) from Folder

In [28]:
from tempfile import TemporaryFile
import numpy as np
import cv2
import os
from skimage import io
from PIL import Image


def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = os.path.join(folder, filename)
        if img is not None:
            images.append(img)
    return images

# Function to load the dictionary and convert to patches

In [29]:
dictionary1 = dictionary.T + mean

file = dictionary1
print(file.shape)

K = file.shape[1]
# file1 = file.T
# print(file1.shape)

i = 0
k = 0
a = np.zeros((8, 16))
b = np.zeros((8, 8))
result = []
#
while k < file.shape[1]:
    arr = file[:,k]
    # a, b = split_array(arr)
    a = np.reshape(arr, (-1, 8))
    # b = np.reshape(b, (-1, 8))

    result.append(a)

    k += 1


(64, 128)


# Save each extracted patch as an image

In [ ]:
count = 0

for i in range(result.__len__()):
    path = 'jup_images'
    io.imsave(os.path.join(path, 'img_{0}.jpg').format(i), result[i])
    count += 1

print(count)

# Load the images and concatenate them to save as a single image

In [ ]:
image = io.imread("C:/Users/DELL/PycharmProjects/ksvd_final/jup_images/img_0.jpg")

images = load_images_from_folder("C:/Users/DELL/PycharmProjects/ksvd_final/jup_images/")

BLUE = [0, 0, 0]
final = []

for img in images:
    img = cv2.imread(img)
    de_img= cv2.copyMakeBorder(img,1,1,1,1,cv2.BORDER_CONSTANT,value= BLUE)
    final.append(de_img)
    
count = 0

for i in range(final.__len__()):
    path = 'jup_images_final'
    io.imsave(os.path.join(path, 'img_{0}.jpg').format(i), final[i])
    count += 1

print(count)

final_images = load_images_from_folder("C:/Users/DELL/PycharmProjects/ksvd_final/jup_images_final/")

new_im = Image.new('RGB', (80, 160))

x_offset = 0
y_offset = 0
count = 0

i = 0
k = 0
j = 0

# for img in images:
#     imga = io.imread(img)
#     count += 1
#     print(count)
#     io.imshow(imga)
#     io.show()


while j < 16:
    while i < 8:
        # fin = open(image)  # open the file
        img = Image.open(final_images[k])
        ext_img = img
        new_im.paste(ext_img, (x_offset, y_offset))
        x_offset += img.size[0]
        i += 1
        print('yes x_{0}'.format(i))
        print('yes k_{0}'.format(k))
        print("Offset x: ", x_offset)
        k += 1
    i = 0
    y_offset += img.size[1]
    x_offset = 0
    j += 1
    print('yes y_{0}'.format(j))
    print("Y-offset", y_offset)

print("X:", x_offset)
print("Y:", y_offset)

new_im.save('test2.jpg')

imagee = io.imread("test2.jpg")
io.imshow(imagee)
io.show()

# Reconstruction of the Image

In [ ]:
def convert_to_uint8(image_in):
    temp_image = np.float64(np.copy(image_in))
    cv2.normalize(temp_image, temp_image, 0, 255, cv2.NORM_MINMAX, dtype=-1)

    return temp_image.astype(np.uint8)

reduced = (gamma.dot(dictionary)) + mean.T

for ele in reduced:
    if ele.any() < 0:
        print("yes")
print("reconstruction patches shape:", reduced.shape)
red = reduced.reshape(res.shape)
print('Final patch shape', red.shape)
final_patches = vector_to_matrix_3d(reduced, -1,p_h,p_w)
reduced_img = reconstruct_patches(final_patches, img.shape, 1)
print("recon image shape:", reduced_img.shape)
# print(reduced)
fin_recon_image = convert_to_uint8(reduced_img)
plt.imsave('sigma_10_sparsity_30.jpg', fin_recon_image, cmap = 'gray')
io.imshow(fin_recon_image, cmap = 'gray')
io.show()
print(reduced_img)
print("Done....")

# If stride > 1, image contrast reduces, to improve the contrast apply Histogram matching with noisy image. 

## Function to implement the histogram matching

In [ ]:
import numpy as np
import cv2

def hist_match(source, template):
    oldshape = source.shape
    source = source.ravel()
    template = template.ravel()

    # get the set of unique pixel values and their corresponding indices and
    # counts
    s_values, bin_idx, s_counts = np.unique(source, return_inverse=True,
                                            return_counts=True)
    t_values, t_counts = np.unique(template, return_counts=True)

    # take the cumsum of the counts and normalize by the number of pixels to
    # get the empirical cumulative distribution functions for the source and
    # template images (maps pixel value --> quantile)
    s_quantiles = np.cumsum(s_counts).astype(np.float64)
    s_quantiles /= s_quantiles[-1]
    t_quantiles = np.cumsum(t_counts).astype(np.float64)
    t_quantiles /= t_quantiles[-1]

    # interpolate linearly to find the pixel values in the template image
    # that correspond most closely to the quantiles in the source image
    interp_t_values = np.interp(s_quantiles, t_quantiles, t_values)

    return interp_t_values[bin_idx].reshape(oldshape)




## Reading both the images and calling the matching function

In [ ]:
from matplotlib import pyplot as plt
from scipy.misc import ascent
import imageio as ios

# Image Input
image = cv2.imread("final_output/reduced_final_20.jpg", 0)

image1 = cv2.imread("girl_face_noise_20.jpg", 0)

x, y= image.shape

x1, y1 = image1.shape


moddedimage = np.zeros([x, y], dtype=np.uint8)

# gray levels
gray_levels = 256
# row times column
mn = x * y

source = image
template = image1
matched = hist_match(source, template)


def ecdf(x):
    vals, counts = np.unique(x, return_counts=True)
    ecdf = np.cumsum(counts).astype(np.float64)
    ecdf /= ecdf[-1]
    return vals, ecdf


x1, y1 = ecdf(source.ravel())
x2, y2 = ecdf(template.ravel())
x3, y3 = ecdf(matched.ravel())


plt.imshow(source, cmap='gray')
plt.axis('off')
plt.title('Source')
plt.show()
plt.imshow(template, cmap='gray')
plt.axis('off')
plt.title('Noisy Image')
plt.show()
plt.imshow(matched, cmap='gray')
plt.axis('off')
plt.title('Denoised Image')
plt.show()

plt.imsave("final_output_match/matched_20.png",matched)

# image = cv2.imread("C:/Users/DELL/PycharmProjects/ksvd_final/matched.png", 0)
# cv2.imshow("image", image)
# cv2.waitKey(0)


In [ ]:
import math

# print(matched.dtype)

# data = matched
# data = data/(data.max())
# data = 255 * data
# img = data.astype(np.uint8)

def convert_to_uint8(image_in):
    temp_image = np.float64(np.copy(image_in))
    cv2.normalize(temp_image, temp_image, 0, 255, cv2.NORM_MINMAX, dtype=-1)

    return temp_image.astype(np.uint8)

# count =0
# for ele in matched:
#     if ele.any() < 0:
#         count += 1

# print(count)

original = cv2.imread("original.jpg",0)
# print(original.dtype)
plt.title("Original")
plt.imshow(original, cmap ='gray')
plt.show()

image = cv2.imread("sigma_10_sparsity_30.jpg", 0)
contrast = convert_to_uint8(image)
# print(original.dtype)
plt.title("Reconstructed")
plt.imshow(contrast, cmap ='gray')
plt.show()


# noisy = cv2.imread("Input Image with noise 10.jpg",0)
# plt.imshow(noisy, cmap = 'gray')
# plt.title("Nosiy")
# plt.show()


# contrast = img
# plt.imshow(contrast, cmap = 'gray')
# plt.show()

i_h, i_w = original.shape
pixels = i_h * i_w

print(original.min())
print(original.max())
print("------------------")
print(contrast.min())
print(contrast.max())

def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2 )
    print("Mse:", mse)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    result_psnr = (20 * math.log10(PIXEL_MAX / math.sqrt(mse)))
    return result_psnr

d=psnr(original,contrast)
print("PSNR between original and reconstructed image:", d)

# d= psnr(original, noisy)
# print("PSNR between original and noisy image:", d)

In [ ]:
# imagee = io.imread("F:/Internship/Data/brain_0% noise/brain_0_105.jpg")
# io.imshow(imagee)
# io.show()
# print(imagee.shape)

image = cv2.imread('final_output/reduced_final_8.jpg', 0)
print(image.shape)
io.imshow(image)
io.show()

image = cv2.imread('girl_face_noise_20.jpg', 0)
print(image.shape)
io.imshow(image)
io.show()

In [ ]:
from skimage.measure import compare_ssim as ssim

def mse(x, y):
    return np.linalg.norm(x - y)
  
image = cv2.imread("input Image.jpg", 0)
recon_image = cv2.imread("sigma_10_sparsity_30.jpg",0)

print(recon_image.shape)
print(image.shape)

mse_noise = mse(image, recon_image)
ssim_noise = ssim(image, recon_image,
                  data_range=recon_image.max() - recon_image.min())

print(mse_noise)
print(ssim_noise)